In [1]:
from pathlib import Path

import pandas as pd
import geopandas as gpd
from tqdm import tqdm

# Load surveys

In [2]:
raw_dir = Path('../data/dhs/raw')

In [4]:
GEO_COLS = ['DHSYEAR', 'DHSCLUST', 'URBAN_RURA', 'LATNUM', 'LONGNUM', 'DATUM', 'geometry', 'yes', 'no']
SURVEY_COLUMNS = ['hv001', 'hv206']
hr_dfs = dict()

for year_dir in tqdm(raw_dir.iterdir()):
    if year_dir.is_dir():
        year = year_dir.name[3:7]
        for dataset_type_dir in year_dir.iterdir():
            dataset_type = dataset_type_dir.stem[2:4]
            if dataset_type == 'HR':
                for file in dataset_type_dir.iterdir():
                    if file.suffix == '.DTA':
                        survey_df = pd.read_stata(file, columns=SURVEY_COLUMNS)
            if dataset_type == 'GE':
                for file in dataset_type_dir.iterdir():
                    if file.suffix == '.shp':
                        geo_df = gpd.read_file(file)
        survey_df['hv206'] = survey_df['hv206'].str.lower()
        survey_df = survey_df.groupby('hv001')['hv206'].value_counts().unstack()
        df = geo_df.merge(survey_df, how='outer', left_on='DHSCLUST', right_on='hv001')
        #df[GEO_COLS].to_file(f'{year}.shp')
        hr_dfs[year] = df[GEO_COLS].copy()

7it [00:09,  1.30s/it]


`hv001` is cluster, `hv206` is whether or not there is electricity.

In [6]:
df

,DHSID,DHSCC,DHSYEAR,DHSCLUST,CCFIPS,ADM1FIPS,ADM1FIPSNA,ADM1SALBNA,ADM1SALBCO,ADM1DHS,...,SOURCE,URBAN_RURA,LATNUM,LONGNUM,ALT_GPS,ALT_DEM,DATUM,geometry,no,yes
0,NG201000000001,NG,2010.0,1.0,NI,NULL,NULL,NULL,NULL,180.0,...,GPS,R,6.732517,8.002321,120.0,94.0,WGS84,POINT (8.00232 6.73252),26.0,NaN
1,NG201000000002,NG,2010.0,2.0,NI,NULL,NULL,NULL,NULL,180.0,...,GPS,R,7.247508,8.657348,150.0,124.0,WGS84,POINT (8.65735 7.24751),25.0,NaN
2,NG201000000003,NG,2010.0,3.0,NI,NULL,NULL,NULL,NULL,180.0,...,GPS,R,7.358567,8.489307,173.0,152.0,WGS84,POINT (8.48931 7.35857),26.0,NaN
3,NG201000000004,NG,2010.0,4.0,NI,NULL,NULL,NULL,NULL,180.0,...,GPS,R,6.864353,8.890684,132.0,114.0,WGS84,POINT (8.89068 6.86435),24.0,2.0
4,NG201000000005,NG,2010.0,5.0,NI,NULL,NULL,NULL,NULL,180.0,...,GPS,R,6.950979,8.345339,160.0,141.0,WGS84,POINT (8.34534 6.95098),NaN,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,NG201000000236,NG,2010.0,236.0,NI,NULL,NULL,NULL,NULL,210.0,...,GPS,R,7.383139,3.964110,211.0,199.0,WGS84,POINT (3.96411 7.38314),NaN,26.0
235,NG201000000237,NG,2010.0,237.0,NI,NULL,NULL,NULL,NULL,210.0,...,GPS,U,7.385056,3.894900,237.0,209.0,WGS84,POINT (3.89490 7.38506),4.0,22.0
236,NG201000000238,NG,2010.0,238.0,NI,NULL,NULL,NULL,NULL,210.0,...,GPS,U,7.407212,3.906515,250.0,212.0,WGS84,POINT (3.90652 7.40721),NaN,26.0
237,NG201000000239,NG,2010.0,239.0,NI,NULL,NULL,NULL,NULL,210.0,...,GPS,U,8.127695,4.241894,348.0,331.0,WGS84,POINT (4.24189 8.12770),3.0,22.0
